Training Video Classification

**Note:** 
Just run this notebook alone
Setup.ipynb and training videos needs to be changed before running all notebooks

In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from skimage.transform import resize 

Using TensorFlow backend.


In [2]:
# read in training frames  
train = pd.read_csv('myVids/train_new.csv')
train

,image,class
0,1_0.mp4_frame0.jpg,0
1,1_0.mp4_frame1.jpg,0
2,1_0.mp4_frame2.jpg,0
3,1_0.mp4_frame3.jpg,0
4,2_0.mp4_frame0.jpg,0
5,2_0.mp4_frame1.jpg,0
6,2_0.mp4_frame10.jpg,0
7,2_0.mp4_frame11.jpg,0
8,2_0.mp4_frame12.jpg,0
9,2_0.mp4_frame13.jpg,0


### What is this dataframe showing? 

This dataframe called "train" has two columns: "image" and "class"
- "image": contains names of each frame 
- "class": contains corresponding class for each frame

##### What does class numbers mean?
0 for good serve   
1 for bad serve 

In this case, **video 1 and 3** is a bad serve and **video 2 and 4** are good serves.


In [3]:
train_image = []

for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 66.36it/s]


(37, 224, 224, 3)

We have 37 images each of size 224, 224 and 3. Next we will create a validation set. 

We need to do **two** things to train our model: 
- train images
- their corresponding model 

In [4]:
y = train["class"]
print(train['class'])

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
Name: class, dtype: int64


### One hot Encoding Classes 

Since we have 2 classes: 0 and 1 (which are numerical data) we need to convert to categorical

[Click me for more info on one hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/)

In [5]:
dummy_y = to_categorical(y)    # one hot encoding Classes

### VGG16 parameters

VGG16 takes an input shape of (224 x 224 x 3) 

Our images are all different sizes so let's reshape them using resize() function. 

In [6]:
image = []
for i in range(0,X.shape[0]): 
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

### Preprocess 

This function preprocess our images so that it fits within the VGG16 model 

In [7]:
X = preprocess_input(X, mode='tf')      # preprocessing the input dat

### Split ya models 
Why do we need to split our models? Because right now we only have training images, we need to have a validation set to check the performance of models on unseen images 

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

### Let's build the model from VGG16

In [9]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 


### Why include_top = False

Fully connected layers at the end can only take fixed size inputs which was probably where we were going wrong 

In [10]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((25, 7, 7, 512), (12, 7, 7, 512))

Shape into single dimensions by below: 

In [11]:
X_train = X_train.reshape(25, 7*7*512)
X_valid = X_valid.reshape(12, 7*7*512)

In [12]:
#normalize me 
train = X_train/X_train.max()      
X_valid = X_valid/X_train.max()

### Steps to setting up model 
1. Build model 
2. Compile model 
3. Train model 

In [13]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid', input_shape=(7*7*512,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


### What does model.summary() show us? 

We have a hidden layer with 1,024 neurons and output layer with 2 neurons(3 classes to predict) 

In [15]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[mcp_save], batch_size=25)

Train on 25 samples, validate on 12 samples
Epoch 1/100
25/25 [==============================] - 0s 19ms/step - loss: 0.6485 - accuracy: 0.7600 - val_loss: 0.7200 - val_accuracy: 0.8333
Epoch 2/100
25/25 [==============================] - 0s 16ms/step - loss: 1.0305 - accuracy: 0.7600 - val_loss: 0.6718 - val_accuracy: 0.8333
Epoch 3/100
25/25 [==============================] - 0s 16ms/step - loss: 0.9587 - accuracy: 0.7600 - val_loss: 0.4732 - val_accuracy: 0.8333
Epoch 4/100
25/25 [==============================] - 0s 16ms/step - loss: 0.6360 - accuracy: 0.7600 - val_loss: 0.6603 - val_accuracy: 0.8333
Epoch 5/100
25/25 [==============================] - 0s 18ms/step - loss: 0.6678 - accuracy: 0.7600 - val_loss: 0.8213 - val_accuracy: 0.1667
Epoch 6/100
25/25 [==============================] - 0s 17ms/step - loss: 0.7962 - accuracy: 0.2400 - val_loss: 0.5385 - val_accuracy: 0.8333
Epoch 7/100
25/25 [==============================] - 0s 17ms/step - loss: 0.5820 - accuracy: 0.7600 - va

Epoch 58/100
25/25 [==============================] - 0s 17ms/step - loss: 0.5470 - accuracy: 0.7600 - val_loss: 0.4558 - val_accuracy: 0.8333
Epoch 59/100
25/25 [==============================] - 0s 16ms/step - loss: 0.5475 - accuracy: 0.7600 - val_loss: 0.4577 - val_accuracy: 0.8333
Epoch 60/100
25/25 [==============================] - 0s 17ms/step - loss: 0.5468 - accuracy: 0.7600 - val_loss: 0.4629 - val_accuracy: 0.8333
Epoch 61/100
25/25 [==============================] - 0s 16ms/step - loss: 0.5462 - accuracy: 0.7600 - val_loss: 0.4682 - val_accuracy: 0.8333
Epoch 62/100
25/25 [==============================] - 0s 17ms/step - loss: 0.5466 - accuracy: 0.7600 - val_loss: 0.4690 - val_accuracy: 0.8333
Epoch 63/100
25/25 [==============================] - 0s 16ms/step - loss: 0.5467 - accuracy: 0.7600 - val_loss: 0.4647 - val_accuracy: 0.8333
Epoch 64/100
25/25 [==============================] - 0s 16ms/step - loss: 0.5460 - accuracy: 0.7600 - val_loss: 0.4596 - val_accuracy: 0.8333

### What does this show us? 

Accuray of 68%

Why so low? Because setup.ipynb needs to be changed because videos are all "good"